In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV,cross_val_score,learning_curve,validation_curve
from sklearn.datasets.samples_generator import make_classification
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from scipy.stats import pearsonr,randint
from sklearn.datasets import load_iris,load_digits
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.datasets.samples_generator import make_blobs
from sklearn.decomposition import PCA
from sklearn import svm, datasets
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

sklearn官方文档：https://scikit-learn.org/stable/index.html  
sklearn中文文档：http://sklearn.apachecn.org/#/

当我们拿到一批原始的数据首先要明确有多少特征，哪些是连续的，哪些是类别的。  
检查有没有缺失值，对确实的特征选择恰当方式进行弥补，使数据完整。  
对连续的数值型特征进行标准化，使得均值为0，方差为1。  
对类别型的特征进行one-hot编码。  
将需要转换成类别型数据的连续型数据进行二值化。  
为防止过拟合或者其他原因，选择是否要将数据进行正则化。  
在对数据进行初探之后发现效果不佳，可以尝试使用多项式方法，寻找非线性的关系。  
根据实际问题分析是否需要对特征进行相应的函数转换。  
准备好美美的数据将为我们寻找美美的模型如虎添翼  

# 数据清理

## 缺失值处理

由于不同的原因，许多现实中的数据集都包含有缺失值，要么是空白的，要么使用NaNs或者其它的符号替代。这些数据无法直接使用scikit-learn分类器直接训练，所以需要进行处理。幸运地是，sklearn中的Imputer类提供了一些基本的方法来处理缺失值，如使用均值、中位值或者缺失值所在列中频繁出现的值来替换。也可使用pandas进行数据的这种处理

In [22]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit([[1, 2], [np.nan, 3], [7, 6]])
# Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
X = [[np.nan, 2], [6, np.nan], [7, 6]]
imp.transform(X)

# Imputer类同样支持稀疏矩阵
# X = sp.csc_matrix([[1, 2], [0, 3], [7, 6]])
# imp = Imputer(missing_values=0, strategy='mean', axis=0)
# imp.fit(X)
# X_test = sp.csc_matrix([[0, 2], [6, 0], [7, 6]])
# imp.transform(X_test)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

array([[ 4.        ,  2.        ],
       [ 6.        ,  3.66666667],
       [ 7.        ,  6.        ]])

## 噪声处理

# 数据集成

# 数据变换

## 标准化

公式：(X-X_mean)/X_std 也叫z-score规范化（零均值规范化）,变换后各维特征有0均值，单位方差。  
sklearn.preprocessing.scale(X, axis=0, with_mean=True,with_std=True,copy=True)  
实际应用中，需要做特征标准化的常见情景：SVM'''  

In [2]:
# 常规计算
X = np.array([[ 1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
(X-X_mean)/X_std

# 使用reprocessing.scale()
X_scale = preprocessing.scale(X)
# X_scale.mean(axis=0) #标准化之后各列的均值为0
# X_scale.std(axis=0) #标准化之后各列的方差为1

# 一般会把train和test集放在一起做标准化，或者在train集上做标准化后，用同样的标准化器去标准化test集，此时可以用StandardScaler
# scaler = preprocessing.StandardScaler().fit(train)
# scaler.transform(train)
# scaler.transform(test)

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

array([ 0.,  0.,  0.])

array([ 1.,  1.,  1.])

## 缩放至某一范围

**1、极大极小化：preprocessing.MinMaxScaler**    
最小-最大规范化对原始数据进行线性变换，变换到[0,1]区间（也可以是其他固定最小最大值的区间）  
使用这种方法的目的包括：1、对于方差非常小的属性可以增强其稳定性；2、维持稀疏矩阵中为0的条目。  

In [3]:
X = np.array([[ 1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit_transform(X)

array([[ 0.5       ,  0.        ,  1.        ],
       [ 1.        ,  0.5       ,  0.33333333],
       [ 0.        ,  1.        ,  0.        ]])

**2、绝对值最大标准化：MaxAbsScaler **   
数据会被规模化到[-1,1]之间。特征中所有数据都会除以最大值。  
这个方法对那些已经中心化均值为0或者稀疏的数据有意义。  

In [4]:
X = np.array([[ 1., -1.,  2.],
              [ 2.,  0.,  0.],
              [ 0.,  1., -1.]])
max_abs_scaler = preprocessing.MaxAbsScaler()
max_abs_scaler .fit_transform(X)

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

## 正则化

正则化的过程是将每个样本缩放到单位范数(每个样本的范数为1)，如果要使用如二次型(点积)或者其它核方法计算两个样本之间的相似性这个方法会很有用。该方法是文本分类和聚类分析中经常使用的向量空间模型（Vector Space Model)的基础。主要思想是对每个样本计算其p-范数，然后对该样本中每个元素除以该范数，使得每个处理后样本的p-范数(l1-norm,l2-norm)等于1。

In [7]:
# sklearn.preprocessing.normalize()函数
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
preprocessing.normalize(X, norm='l2') #默认L2范数

# 使用sklearn.preprocessing.Normalizer()类
normalizer = preprocessing.Normalizer().fit(X)  # fit does nothing
normalizer.transform(X)

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

## 二值化

**preprocessing.Binarizer(threshold=0)：可以设置threshold阈值，结果数据值大于阈值的为1，小于阈值的为0**  
特征二值化是将数值型的特征值转换为布尔值，可以用于概率估计。这在文本处理过程中也非常常见。   

In [8]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
binarizer = preprocessing.Binarizer().fit(X)
binarizer.transform(X)

array([[ 1.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

## 类别特征one-hot编码

当某些特征不是连续取值而是分类数据时，就需要对分类特征进行编码，比如人的性别有[“男”, “女”]之分，国籍可以是[“中国”, “英国”, “美国”]。这样的特征可以分别用不同的整数进行标记，比如[“男”, “女”]分表表示成[0, 1]，[“中国”, “英国”, “美国”]分别表示成[0, 1, 2]，但是，这种整数表示方法在模型中会被理解成不同的大小（比如2和0的距离比1和0大）。解决这一问题的一个方法是使用one-of-K或者one-hot编码，通过OneHotEncoder实现。这一估计量将每个含有m个取值的分类特征转化为m个二值特征，其中只有一个处于active状态。  
两种哑编码的实现方法，pandas和sklearn。区别是pandas默认只处理字符串类别变量，sklearn默认只处理数值型类别变量(需要先 LabelEncoder )

###  pd.get_dummies

pd.get_dummies(prefix=)  
pandas的get_dummies()可以直接对变量进行one-hot编码，其中prefix是为one-hot编码后的变量进行命名。

### OneHotEncoder

OneHotEncoder不能直接对字符型变量编码，因此我们需要先将字符型变量通过LabelEncoder转换为数值型变量，如果本身为数值型则不用。    

In [9]:
X = [[ 0, 0, 3],
     [ 1, 1, 0],
     [ 0, 2, 1],
     [ 1, 0, 2]]
nc = preprocessing.OneHotEncoder().fit(X)
nc.transform([[0,1,3]]).toarray()
# array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])  # 一共9位
# >>> # feature1只有0,1两个取值，因此是两位
# >>> # feature2有0,1,2三个取值，因此是三位
# >>> # feature3有0,1,2,3四个取值，因此是四位

array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

每个特征的分类个数默认上根据输入数据集自动计算。但是也可以通过设置参数n_values进行人为设定。在上述例子中，三个分类属性的可选值数量分别为2，3，4。特别是当训练数据集在某个可能取值上没有训练样本时，需要人为制定分类数量。

In [10]:
enc = preprocessing.OneHotEncoder(n_values=[2, 3, 4])
enc.fit([[1,2,3],[0,2,0]])
enc.transform([[1 , 0, 0]]).toarray()
#array([[ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.]])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values=[2, 3, 4], sparse=True)

array([[ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.]])

## 类别特征LabelEncoder标签编码

https://blog.csdn.net/jin_tmac/article/details/80611676

In [17]:
le = preprocessing.LabelEncoder()
le.fit([1, 2, 2, 6])
le.transform([1, 1, 2, 6]) #array([0, 0, 1, 2])
# #非数值型转化为数值型
le.fit(["amsterdam","paris","paris","tokyo" ])
le.transform(["tokyo","tokyo","paris"]) #array([2, 2, 1])

LabelEncoder()

array([0, 0, 1, 2], dtype=int64)

LabelEncoder()

array([2, 2, 1], dtype=int64)

## 类别特征人工处理

In [ ]:
categorical_cols = ['grade_id','enter_type','city_id','spr_code1','spr_code2','spr_code3']
# 保存类别变量映射字典
def save_categorical_dic(data):
    categorical_dic = {}
    for i in categorical_cols:
        categorical_cnt = data[i].value_counts(normalize=True)
        categorical_index = categorical_cnt[categorical_cnt>0.01].index.tolist()
        categorical_index.append('others')
        categorical_dic[i] = dict(zip(categorical_index,range(len(categorical_index))))
    with open('categorical_dic.txt','w') as f:
        f.write(str(categorical_dic))

# 获取类别变量映射字典       
def get_categorical_dic():
    with open('categorical_dic.txt','r') as f:
        a = f.read()
        return eval(a)

# 类别变量设置
def set_categorical(data):
    categorical_dic = get_categorical_dic()
    for i in categorical_cols:
        data[i] = data[i].map(lambda x:categorical_dic[i][x] if x in categorical_dic[i] else list(categorical_dic[i].values())[-1])
    return data   

# save_categorical_dic(data_train)
data_train = set_categorical(data_train)

## 生成多项式特征

在输入数据存在非线性特征时，这一操作对增加模型的复杂度十分有用。一种常见的用法是生成多项式特征，能够得到特征的高阶项和相互作用项。多项式特征经常用于使用多项式核函数的核方法（比如SVC和KernelPCA）

In [26]:
# 特征向量X=(X1, X2)被转化为(1, X1, X2, X1^2, X1X2, X2^2)
X=[[0, 1],
   [2, 3],
   [4, 5]]
poly = preprocessing.PolynomialFeatures(2)
poly.fit_transform(X)

# 通过设定interaction_only=True仅实现相互作用项，X=(X1, X2, X3)被转化为to (1, X1, X2, X3, X1X2, X1X3, X2X3, X1X2X3)
X=[[0, 1, 2],
   [3, 4, 5],
   [6, 7, 8]]
poly = preprocessing.PolynomialFeatures(degree=3, interaction_only=True)
poly.fit_transform(X)

array([[  1.,   0.,   1.,   0.,   0.,   1.],
       [  1.,   2.,   3.,   4.,   6.,   9.],
       [  1.,   4.,   5.,  16.,  20.,  25.]])

array([[   1.,    0.,    1.,    2.,    0.,    0.,    2.,    0.],
       [   1.,    3.,    4.,    5.,   12.,   15.,   20.,   60.],
       [   1.,    6.,    7.,    8.,   42.,   48.,   56.,  336.]])

# 数据规约